<h1> <center> M9 Lab 3: Assessment of Weather API Call

# Acquire an API Key from OpenWeatherMap
- First, register on the OpenWeatherMap website and create an account.
- Next, locate and copy the API key from your account dashboard.

# Python Function for pulling weather Data

In [1]:
import requests
import json
import pandas as pd

def get_weather_data(city, api_key):
    try:
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {'q': city, 'appid': api_key}
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an exception for HTTP errors (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        # Handle request exceptions such as connection errors, timeouts, etc.
        raise Exception("Failed to fetch weather data:", e)

## Installing Required Libraries

In [2]:
!pip install boto3

## Uploading data to S3 bucket

In [3]:
import boto3
import json

def uploading_to_s3(bucket_name, file_name, data):
    try:
        s3 = boto3.resource('s3')
        obj = s3.Object(bucket_name, file_name)
        obj.put(Body=json.dumps(data))
        print(f"Data uploaded successfully to s3://{bucket_name}/{file_name}")
    except Exception as e:
        print("Failed to upload data to S3:", e)

In [4]:
# Main execution
api_key = "e78b075bae5ac2aaef09f2a9dbdcce08"   # Replace with your API key
city = "Ohio"  # Replace with desired city
bucket_name = "lab-03-group1"  # Replace with your S3 bucket name
file_name = "Group_1_weather_data_ohio.json"


In [5]:
# Get weather data
weather_data = get_weather_data(city, api_key)

# Upload data to S3
uploading_to_s3(bucket_name, file_name, weather_data)


Data uploaded successfully to s3://lab-03-group1/Group_1_weather_data_ohio.json


In [6]:
try:
    weather_data = get_weather_data(city, api_key)
    print(weather_data)
except Exception as e:
    print(f"An error occurred: {e}")

{'coord': {'lon': -83.0002, 'lat': 40.2503}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 286.93, 'feels_like': 286.82, 'temp_min': 286.03, 'temp_max': 287.64, 'pressure': 1012, 'humidity': 94}, 'visibility': 10000, 'wind': {'speed': 2.24, 'deg': 180, 'gust': 3.58}, 'rain': {'1h': 0.45}, 'clouds': {'all': 100}, 'dt': 1712737132, 'sys': {'type': 2, 'id': 2084058, 'country': 'US', 'sunrise': 1712746830, 'sunset': 1712793923}, 'timezone': -14400, 'id': 5165418, 'name': 'Ohio', 'cod': 200}


## Converting JSON format to CSV format

In [7]:
import boto3

def print_bucket_contents(bucket):
    try:
        # Create an S3 client
        s3 = boto3.client('s3')
        
        # List objects in the specified bucket
        response = s3.list_objects_v2(Bucket=bucket)

        # Check if the bucket contains any objects
        if 'Contents' in response:
            print("Contents of the '{}' bucket:".format(bucket))
            for obj in response['Contents']:
                print(obj['Key'])
        else:
            print("The bucket '{}' is empty.".format(bucket))
    
    except Exception as e:
        print(f"An error occurred while listing objects in bucket '{bucket}': {e}")


bucket_name = "lab-03-group1"
print_bucket_contents(bucket_name)


Contents of the 'lab-03-group1' bucket:
Group_1_weather_data_london.csv
Group_1_weather_data_london.json
Group_1_weather_data_ohio.json


In [8]:
# Configure to display all the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Converting json to dataframe
flattened_data = pd.json_normalize(weather_data) # Flatten the JSON data

# Flatten 'weather' column separately with prefix
weather_data_norm = pd.json_normalize(weather_data['weather']).add_prefix('weather.')

# Merge the two DataFrames
df = pd.concat([flattened_data, weather_data_norm], axis=1)
df = df.drop(columns = ['weather'], axis = 1) # drop the weather column

# Display DataFrame
df

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,wind.gust,rain.1h,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,weather.id,weather.main,weather.description,weather.icon
0,stations,10000,1712737132,-14400,5165418,Ohio,200,-83.0002,40.2503,286.93,286.82,286.03,287.64,1012,94,2.24,180,3.58,0.45,100,2,2084058,US,1712746830,1712793923,500,Rain,light rain,10n


In [9]:
import boto3
import pandas as pd

# Assume 'df' is the DataFrame containing weather data

file_name_csv = "Group_1_weather_data_ohio.csv"

# Converting DataFrame to CSV string data
try:
    csv_content = df.to_csv(index=False)
except Exception as e:
    print("Error while converting DataFrame to CSV Format:", e)
    csv_content = None

# Uploading the CSV content to S3 bucket
if csv_content is not None:
    try:
        s3 = boto3.client('s3')
        response = s3.put_object(
            Bucket=bucket_name,
            Key=file_name_csv,
            Body=csv_content.encode('utf-8')  # Convert to bytes
        )
        print("Data uploaded successfully to S3 bucket")
    except Exception as e:
        print("Error uploading file to S3:", e)
else:
    print("CSV content is None, Upload Failed.")


Data uploaded successfully to S3 bucket


In [10]:
import boto3

def check_json_file(bucket_name, file_name):
    try:
        s3 = boto3.client('s3')
        response = s3.head_object(Bucket=bucket_name, Key=file_name)
        print(f"JSON file '{file_name}' exists in the '{bucket_name}' bucket.")
    except Exception as e:
        print(f"JSON file '{file_name}' does not exist in the '{bucket_name}' bucket. Error: {e}")

# Usage
check_json_file(bucket_name, file_name)

JSON file 'Group_1_weather_data_ohio.json' exists in the 'lab-03-group1' bucket.


In [11]:
print_bucket_contents(bucket_name)

Contents of the 'lab-03-group1' bucket:
Group_1_weather_data_london.csv
Group_1_weather_data_london.json
Group_1_weather_data_ohio.json
